<img src="../../shared/img/banner.svg"></img>

# Lab 03 - Models and Monsters

In [ ]:
%matplotlib inline

In [ ]:
import sys

sys.path.append("../../")

from shared.src import quiet
from shared.src import seed

In [ ]:
import random

from IPython.display import Image
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pymc3 as pm
import seaborn as sns

import shared.src.utils.util as util

## Learning Objectives


1. Practice building models of random processes.
1. Learn to recognize problems in real life that you can use models to solve.

## Distributions & Dragons

In [ ]:
Image("./img/dndlogo.jpg", width=500)

[*Dungeons & Dragons*](http://dnd.wizards.com/) (R), or D&D, is the world's most popular tabletop roleplaying game.
In addition to non-random, human-controlled elements of storytelling, it features lots of cases where story outcomes are determined by the rolling of dice of varying sides, most prominently a 20-sided die.

One of the most important examples is *attacking*, when one character tries to use a weapon against another.
The core idea is as follows:

1. The attacker rolls a 20 sided die (`pm.DiscreteUniform`, with `lower=1` and `upper=20`).
1. If the number on that die is as high or higher than the armor of the opponent, the attack "hits" and deals damage.
1. If the number on that die is also as high or higher than the "critical range" of the attacker's weapon, then the attacker rolls another 20 sided die. This is called a "confirmation roll".
1. If that number is also higher than the armor of the opponent, the attacker deals a "critical hit", meaning the attacker deals damage again.

A character is dead if the amount of damage dealt exceeds their health points,
so being able to do more damage to enemies is better.

Unlike in the case of games like
[*Risk*](https://en.wikipedia.org/wiki/Risk_(game)) or
[*Root*](https://boardgamegeek.com/boardgame/237182/root) it can be much harder in D&D to use math to calculate things like how much damage a weapon will do on average or in the median (especially once additional rules get added to the above!).

However, it's not that much harder to *simulate*!
The function below produces models for how attacks are carried out in D&D.

In [ ]:
def construct_dnd_model(function_to_add_damage, armor=15):
    dnd_model = pm.Model()

    with dnd_model:

        attack_roll = pm.DiscreteUniform(name="attack_roll",
                                         lower=1, upper=20)
        confirmation_roll = pm.DiscreteUniform(name="confirmation_roll",
                                               lower=1, upper=20)

        attack_hit = pm.Deterministic(name="attack_hit",
                                      var=attack_roll >= armor)

        base_damage_roll, critical_damage_roll, critical_threat = function_to_add_damage(attack_roll)

        critical_confirmed = pm.Deterministic(name="critical_confirmed",
                                              var=confirmation_roll >= armor)

        critical_hit = pm.Deterministic(name="critical_hit",
                                        var=critical_threat * critical_confirmed)

        total_damage = pm.Deterministic(name="total_damage",
                                        var=attack_hit * base_damage_roll + critical_hit * critical_damage_roll)
        
    return dnd_model

Note that the simulation above is not an exact step-for-step implementation of the process described by the rules.
For example, instead of, e.g., only doing the `confirmation_roll` when the `attack_roll` is above the `critical_range`, we always do it, but we only add in the damage if `critical_hit` is `True`.

Because different weapons do damage differently, we need to provide a `function_to_add_damage` to complete our model.
The template below shows the basic outline of such a function.

```python
def function_to_add_damage(attack_roll):
    """Differs based on the weapon, which have varying damage rolls and critical ranges.
    """
    critical_range = ...
    
    base_damage_roll = ...
    critical_damage_roll = ...
    critical_threat = ...
    
    return base_damage_roll, critical_damage_roll, critical_threat
```

An example weapon is below.
In D&D terms, the longsword would be described as dealing damage with _two six-sided die_
(notice the `base_damage_roll` adds together two `DiscreteUniform` random variables between `1` and `6`)
and having a _critical range of 20_.

In [ ]:
def add_longsword(attack_roll):
    
    critical_range = 20
    base_damage_roll = pm.Deterministic(name="base_damage_roll",
                                        var=pm.DiscreteUniform("bd_d6_1", lower=1, upper=6) +
                                        pm.DiscreteUniform("bd_d6_2", lower=1, upper=6))
    critical_damage_roll = pm.Deterministic(name="critical_damage_roll",
                                            var=pm.DiscreteUniform("cd_d6_1", lower=1, upper=6) +
                                            pm.DiscreteUniform("cd_d6_2", lower=1, upper=6))
    critical_threat = pm.Deterministic(name="critical_threat",
                                       var=attack_roll >= critical_range)
    
    return base_damage_roll, critical_damage_roll, critical_threat
    

The code cells below will construct a model for damage dealing with the longsword, draw 1000 samples from it, and then convert those samples into a dataframe for ease of analysis.

In [ ]:
longsword_model = construct_dnd_model(add_longsword)

In [ ]:
with longsword_model:
    longsword_attacks = pm.sample(draws=1000)

In [ ]:
longsword_df = util.samples_to_dataframe(longsword_attacks)

From the samples, compute the following:

- Fraction of times a critical hit happens
- Average damage, including and not including critical hits
- Fraction of times the attack roll and the confirmation roll are both odd

### A Critical Decision

As a reward for completing a quest, a wizard offers your friend, Cora the Barbarian, a choice of two weapons: a keen katana +1 and a longsword +2.
As the party's datamancer, it is your responsibility to provide data-driven modeling services and support optimal adventuring decision-making.

Help Cora select a weapon by simulating the damage output for both of them.
Produce samples, visualize and analyze them,
and then make an argument for either the katana or the longsword.

Here's how you'll do it:

Write two functions below which add weapon damage rolls and critical threats
to the model:
`add_keen_katana` and
`add_longsword_plus_two`.

The `keen_katana` has a `critical_range` of `16` and rolls two six-sided die and adds `1`.
The `longsword_plus_two` has a `critical_range` of `20` and rolls two six-sided die and adds `2`.

Give those functions to `construct_dnd_model` and then draw a good number of samples from each (maybe just a thousand while you're still playing around, but then maybe a few tens of thousands once you're ready to do your final analysis).

Create visualizations for the resulting data, comparing the damage output of the `longsword_plus_two` and of the `keen_katana`.
Calculate descriptive statistics on the damage, including at least the mean, standard deviation, median, and 25th and 75th percentiles.

Combine your visualizations and your statistics to make an argument to Cora about which weapon she should choose.
If you feel there's no strong reason to prefer either, you're free to claim that as well!
Just make your argument data-driven, rather than about the stylistic differences between a katana and a longsword (you are a _datamancer_, not an emotion lord).

In [ ]:
# def add_keen_katana(attack_roll):
#     return base_damage_roll, critical_damage_roll, critical_threat

# def add_longsword_plus_two(attack_roll):
#     return base_damage_roll, critical_damage_roll, critical_threat